# 第7章　実践編2：腫瘍特異的ネオ抗原の機械学習を用いた予測腫瘍特異的ネオ抗原の機械学習を用いた予測

-長谷川嵩矩

編集部注：2023年5月29日最終更新．コードの一部がお手元の書籍と異なる可能性がございます．正誤・更新情報は弊社ウェブサイトの[本書詳細ページ](https://www.yodosha.co.jp/jikkenigaku/book/9784758122634/index.html)をご参照ください．

##### 入力7-1

In [ ]:
! pip install torchmetrics==0.7.1
! pip install biopython

##### 入力7-2


In [ ]:
%matplotlib inline
import os
import urllib.request
import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional
import torch.optim
from sklearn.metrics import accuracy_score
from torchmetrics.functional import precision_recall
import matplotlib
import matplotlib.pyplot as plt
import Bio
from Bio.Seq import Seq

##### 入力7-3


In [ ]:
# Chr     Start       End Ref Alt Func.refGene Gene.refGene   GeneDetail.refGene ExonicFunc.refGene                              AAChange.refGene   cytoBand depth_tumor variantNum_tumor depth_normal
# 1 116941338 116941338   T   C       exonic       ATP1A1           synonymous                SNV   ATP1A1:NM_001160234:exon16:c.T2127C:p.D709D     1p13.1         100               39          111
# 4  24556416  24556416   T   C       exonic        DHX15        nonsynonymous                SNV        DHX15:NM_001358:exon5:c.A1012G:p.T338A     4p15.2         143               47          151
# 4  70156404  70156404   -   T       exonic      UGT2B28           frameshift          insertion   UGT2B28:NM_053039:exon5:c.1186dupT:p.L395fs     4q13.2          43               15           41
# 6  75899298  75899298   T   -       exonic      COL12A1           frameshift           deletion    COL12A1:NM_004370:exon6:c.628delA:p.I210fs       6q13         122               38           73
# 9  89561162  89561162   C   T       exonic         GAS1        nonsynonymous                SNV          GAS1:NM_002048:exon1:c.G533A:p.R178H    9q21.33          20                5           26
# ...

##### 入力7-4

In [ ]:
normal_rna = Seq("GCATGCTGCATGCATGATGCCATGCATGCTGCATGCTGCATG")
tumor_rna = Seq("GCATGCTGCATGCATGATGCCACGCATGCTGCATGCTGCATG")
print(" 野生型RNA配列:.." + str(normal_rna) + "..")
print("腫瘍特異的RNA配列:.." + str(tumor_rna) + "..")

print(" 野生型タンパク質配列:.." + str(normal_rna.translate()) + "..")
print("腫瘍特異的タンパク質配列:.." + str(tumor_rna.translate()) + "..")

##### 入力7-5

In [ ]:
'''
1. まずはデータをダウンロードする
'''
data_dir = './pepdata/' # データセットの格納先
if not os.path.exists(data_dir): # 指定したフォルダーが存在しない場合は作成する
    os.mkdir(data_dir)

# Hao et al. 2021の論文で使われている公開データセットのダウンロード先
# 今回はHLA Class1 A0301のデータを利用
train_url = 'https://github.com/haoqing12/APPM/raw/master/DATA/train_data/A0301'
test_url = 'https://github.com/haoqing12/APPM/raw/master/DATA/test_data/A0301'

# ファイル名を連結して格納先を指定
train_save_path = os.path.join(data_dir, 'train_A0301.csv')
test_save_path = os.path.join(data_dir, 'test_A0301.csv')

# ダウンロードを実行
urllib.request.urlretrieve(train_url, train_save_path)
urllib.request.urlretrieve(test_url, test_save_path)

print("Stored training-data name is ", train_save_path)
print("Stored test-data name is ", test_save_path)
pd.read_csv(train_save_path, nrows=5).head # データセットの中身を確認


##### 入力7-6

In [ ]:
'''
2. トランスフォーマーオブジェクトの作成
'''
class PepTransform():
    allSequences = 'ACEDGFIHKMLNQPSRTWVYZ' # アミノ酸配列
    char2int = dict((c, i) for i, c in enumerate(allSequences)) # アミノ酸配列にインデックスを付けてdict型に格納
  
    def peptideOneHotMap(self, peptide):
        peptide_integer = [self.char2int[char] for char in peptide] # 入力したペプチドのアミノ酸を上記で定義した番号に変換
        peptide_onehot = list() # OneHotベクトル形式のペプチド配列
        for value in peptide_integer:
            letter = [0 for _ in range(len(self.allSequences))] # 要素が0でlen(self. allSequences)の長さを持つ配列を作成
            letter[value] = 1 # 対応するアミノ酸の要素を1で埋める
            peptide_onehot.append(letter) # 配列をリストに加える
        return np.asarray(peptide_onehot)

    def __init__(self):
        '''インスタンス変数の初期化
        '''

    def __call__(self, peptide):
        '''コールバック関数
        与えられたpeptide配列に対応するOneHotベクトルのリストを返す
        '''
        return self.peptideOneHotMap(peptide)

##### 入力7-7

In [ ]:
'''
3. ペプチドの長さを一定に変更する関数を作成
'''
# ペプチド長を11に固定し，足りない部分にZを埋め込んだペプチドを返す関数を定義する
# 入力はpandas.Seriesであり，listに変換して返す
def complementPeptides(peptides_series):
    peptides_list=peptides_series.tolist()
    for i in range(len(peptides_list)):
        if len(peptides_list[i]) < 11: # 11未満の場合は後ろにZを足す
            peptides_list[i] = peptides_list[i] + 'Z'*(11 - len(peptides_list[i]))
        else:
            peptides_list[i] = peptides_list[i][:11] # 11以上の場合は11番目までを残す
    return peptides_list


#例
peptides_list = ['ACCMHDACCMHDAAA', 'ACCMHDATH']
for i in range(len(peptides_list)):
    if len(peptides_list[i]) < 11:
        peptides_list[i] = peptides_list[i] + 'Z' * (11 - len(peptides_list[i]))
    else:
        peptides_list[i] = peptides_list[i][:11]
    print("Generated Peptide is: " + peptides_list[i])


##### 入力7-8


In [ ]:
'''
4. 学習に用いるペプチドのデータセットを作成するクラス
'''
class MakeDataset(data.Dataset):
    '''
    PyTorchのDatasetクラスを継承
    Attributes:
        save_path: ペプチドのリストを有するテキストファイルの格納先
        transform: 前処理クラスのインスタンス
    Returns:
        X: OneHotベクトルに変換したtorch.Tensor型の学習用入力ファイル
        Y: 正解ラベル(結合親和性の有無)
    '''
    
    def __init__(self, save_path, transform=None):
        '''インスタンス変数の初期化
        '''
        super().__init__() # スーパークラスの__init__()を実行
        
        df = pd.read_csv(save_path, header=0) # データを読み込む
        df = df[df['Peptide'].str.contains('X') == False] # 不明なペプチドが含まれているPeptide配列を削除
        df = df[df['Peptide'].str.contains('B') == False] # 不明なペプチドが含まれているPeptide配列を削除
        df = df[df['Peptide'].str.contains('U') == False] # 不明なペプチドが含まれているPeptide配列を削除
        
        print('{0}のファイル:'.format(save_path))
        print('#Original Negative data is: ' + str(df.loc[df['BindingCategory'] == 0].shape))
        print('#Original Positive data is: ' + str(df.loc[df['BindingCategory'] == 1].shape) + '\n')
        
        # 学習時のみPositiveサンプルのオーバーサンプリングを行う
        if "train" in save_path:
            df_0 = df.loc[df['BindingCategory'] == 0]
            df_1 = df.loc[df['BindingCategory'] == 1]
            df = pd.concat([df_0, df_1, df_1], axis=0) # Positiveサンプルを2倍に増やす
            print('オーバーサンプリング後:')
            print('#Original Negative data is: ' + str(df.loc[df['BindingCategory'] ==0].shape))
            print('#Original Positive data is: ' + str(df.loc[df['BindingCategory'] == 1].shape) + '\n')
        df = df.sample(frac=1).reset_index() # インデックスをリセットする
        
        self.peptides =complementPeptides(df['Peptide']) # OneHotベクトルのペプチド配列
        self.y = df['BindingCategory'] # 正解ラベル
        self.transform = transform # 前処理クラスのインスタンス
    def __len__(self):
        '''len(obj)で実行されたときにコールされる関数
        ここではペプチド総数を返すようにセット
        '''
        return len(self.peptides)

    def __getitem__(self, index):
        '''Datasetクラスの__getitem__()をオーバーライドする
        OneHotベクトルをtorch.Tensor型に変換して返す
        '''

        # index番目のペプチドを取得
        peptide_transformed = self.transform(self.peptides[index])

        X = torch.from_numpy(peptide_transformed.astype(np.float32)).clone() # numpy.ndarrayからtorch.Tensorに変換
        Y = torch.tensor(self.y[index], dtype=torch.long).clone() # 同様にtorch.Tensorに変換
        return torch.reshape(X, (1, X.shape[0], X.shape[1])), Y # PyTorchの入力データは(バッチサイズ(外側から指定)，チャネル数(=1)，配列の次元)


##### 入力7-9

---



In [ ]:
'''
5. データローダーの生成
'''
batch_size = 32    # ミニバッチのサイズを指定

# MakeDatasetで訓練データと正解ラベルを取得
train_dataset = MakeDataset(
    save_path = train_save_path, 
    transform = PepTransform())

# MakeDatasetで検証データと正解ラベルを取得
test_dataset = MakeDataset(
    save_path = test_save_path, 
    transform = PepTransform())

# (32, 1, 11, 21)のtorch.Tensorを生成する訓練データ用のデータローダーを作成
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False)

# (32, 1, 11, 21)のtorch.Tensorを生成する検証データ用のデータローダーを作成
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False)

# データローダーが返すミニバッチの先頭データの形状を出力して確認する
for (x, t) in train_dataloader: # 訓練データ
    print(x.shape)
    print(t.shape)
    break
    
for (x, t) in test_dataloader: # テストデータ
    print(x.shape)
    print(t.shape)
    break

##### 入力7-10

In [ ]:
'''
6. 畳み込みニューラルネットワークの構築
'''
class CNN(nn.Module):
    def __init__(self):
        '''インスタンス変数の初期化
        '''
        super().__init__() # スーパークラスの__init__()を実行

        # 第1層: 畳み込み層1
        # (バッチサイズ,1,11,21) -> (バッチサイズ,128,12,22)
        self.conv1a = nn.Conv2d(in_channels=1, # 入力チャネル数
                                out_channels=128, # 出力チャネル数
                                kernel_size=(2, 2), # フィルターサイズ
                                stride=(1, 1), # スライド幅
                                padding=1, # 周囲に1のパディングを行う
                                padding_mode='zeros') # パディング部分を0で埋める
        self.dropout1a = nn.Dropout(0.40) # Dropuotの設定
        # 畳み込み層へのDropoutの適用は一般的ではないが，既存モデルを踏襲するために設定する
        # 第2層: 畳み込み層2
        # (バッチサイズ,128,12,22) -> (バッチサイズ,128,7,12)
        self.conv1b = nn.Conv2d(in_channels=128, # 入力チャネル数
                                out_channels=128, # 出力チャネル数
                                kernel_size=(2, 2), # フィルターサイズ
                                stride=(2, 2), # スライド幅
                                padding=1, # 周囲に1のパディングを行う
                                padding_mode='zeros') # パディング部分を0で埋める
        self.dropout1b = nn.Dropout(0.40)

        # 第3層: 畳み込み層3
        # (バッチサイズ,128,7,12) -> (バッチサイズ,256,4,7)
        self.conv1c = nn.Conv2d(in_channels=128, # 入力チャネル数
                                out_channels=256, # 出力チャネル数
                                kernel_size=(2, 2), # フィルターサイズ
                                stride=(2, 2), # スライド幅
                                padding=1, # 周囲に1のパディングを行う
                                padding_mode='zeros') # パディング部分を0で埋める
        self.dropout1c = nn.Dropout(0.40)

        # 第4層: 畳み込み層4
        # (バッチサイズ,256,4,7) -> (バッチサイズ,256,5,8)
        self.conv1d = nn.Conv2d(in_channels=256, # 入力チャネル数
                                out_channels=256, # 出力チャネル数
                                kernel_size=(2, 2), # フィルターサイズ
                                stride=(1, 1), # スライド幅
                                padding=1, # 周囲に1のパディングを行う
                                padding_mode='zeros') # パディング部分を0で埋める
        self.dropout1d = nn.Dropout(0.40)

        # 第5層: 畳み込み層5
        # (バッチサイズ,256,5,8) -> (バッチサイズ,256,6,9)
        self.conv1e = nn.Conv2d(in_channels=256, # 入力チャネル数
                                out_channels=256, # 出力チャネル数
                                kernel_size=(2, 2), # フィルターサイズ
                                stride=(1, 1), # スライド幅
                                padding=1, # 周囲に1のパディングを行う
                                padding_mode='zeros') # パディング部分を0で埋める
        self.dropout1e = nn.Dropout(0.40)

        # 第6層: 畳み込み層6
        # (バッチサイズ,256,6,9) -> (バッチサイズ,512,6,4)
        self.conv1f = nn.Conv2d(in_channels=256, # 入力チャネル数
                                out_channels=512, # 出力チャネル数
                                kernel_size=(1, 2), # フィルターサイズ
                                stride=(1, 2), # スライド幅
                                padding=0, # 周囲に1のパディングを行う
                                padding_mode='zeros') # パディング部分を0で埋める
        self.dropout1f = nn.Dropout(0.40)

        # 第7層: 畳み込み層7
        # (バッチサイズ,512,6,4) -> (バッチサイズ,512,6,4)
        self.conv1g = nn.Conv2d(in_channels=512, # 入力チャネル数
                                out_channels=512, # 出力チャネル数
                                kernel_size=(1, 1), # フィルターサイズ
                                stride=(1, 1), # スライド幅
                                padding=0, # 周囲に1のパディングを行う
                                padding_mode='zeros') # パディング部分を0で埋める
        self.dropout1g = nn.Dropout(0.40)

        # 第8層: 畳み込み層8
        # # (バッチサイズ,512,6,4) -> (バッチサイズ,256,6,4)
        self.conv1h = nn.Conv2d(in_channels=512, # 入力チャネル数
                                out_channels=256, # 出力チャネル数
                                kernel_size=(1, 1), # フィルターサイズ
                                stride=(1, 1), # スライド幅
                                padding=0, # 周囲に1のパディングを行う
                                padding_mode='zeros') # パディング部分を0で埋める
        self.dropout1 = nn.Dropout(0.50)

        # 第9層: 全結合層1
        # (バッチサイズ,256*6*4) -> (バッチサイズ,128)
        self.fc1 = nn.Linear(256*6*4, 128)
        # ドロップアウト:
        self.dropout2 = nn.Dropout(0.25)

        # 第10層: 出力層
        # (バッチサイズ,128) -> (バッチサイズ,2)
        self.fc2 = nn.Linear(128, 2)

    # 深層学習モデルを構築する
    def forward(self, x):
        x = torch.relu(self.conv1a(x)) # conv1
        x = self.dropout1a(x)
        x = torch.relu(self.conv1b(x)) # conv2
        x = self.dropout1b(x)
        x = torch.relu(self.conv1c(x)) # conv3
        x = self.dropout1c(x)
        x = torch.relu(self.conv1d(x)) # conv4
        x = self.dropout1d(x)
        x = torch.relu(self.conv1e(x)) # conv5
        x = self.dropout1e(x)
        x = torch.relu(self.conv1f(x)) # conv6
        x = self.dropout1f(x)
        x = torch.relu(self.conv1g(x)) # conv7
        x = self.dropout1g(x)
        x = torch.relu(self.conv1h(x)) # conv8

        x = self.dropout1(x)
        x = x.view(-1, 256*6*4) # フラット化
        x = torch.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        
        return x

##### 入力7-11

In [ ]:
'''
7. モデルのインスタンスを生成
'''
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 使用可能なデバイス (CPUまたはGPU)を取得する
print(device)
model = CNN().to(device) # モデルオブジェクトを生成し，使用可能なデバイスを設定する
model # モデルの構造を出力する

##### 入力7-12

In [ ]:
'''
8. 損失関数とオプティマイザの生成
'''
criterion = nn.CrossEntropyLoss() # クロスエントロピー誤差のオブジェクトを生成
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # オプティマイザを指定

##### 入力7-13

In [ ]:
'''
9. パラメータの更新処理
'''
def train_step(x, target):
    '''パラメータ更新を行う
    '''
    model.train() # モデルを訓練(学習)モードにする
    preds = model(x) # モデルの出力を取得
    pr = precision_recall(preds, target, average='macro', num_classes=2) # precisionとrecallを出力(macroを指定)
    loss = criterion(preds, target) # 出力と正解ラベルの誤差から損失を取得
    optimizer.zero_grad() # 勾配を0で初期化(累積してしまうため)
    loss.backward() # 逆伝播を行うことで勾配を計算する
    optimizer.step() # 設定した最適化方法を適用してパラメータを更新する

    return loss, preds, pr[0], pr[1]

##### 入力7-14

In [ ]:
'''
10. 評価用データセットを用いてモデルの評価を行う関数
'''
def test_step(x, target):
    '''評価用データセットを入力して損失と予測値を返す
    '''
    model.eval() # モデルを評価モードにする
    preds = model(x) # モデルの出力を取得
    pr = precision_recall(preds, target, average='macro', num_classes=2) # precisionとrecallを出力(macroを指定)
    loss = criterion(preds, t) # 出力と正解ラベルの誤差から損失を取得

    return loss, preds, pr[0], pr[1]

##### 入力7-15

In [ ]:
'''
11. モデルを使用して学習する
'''
epochs = 120 # エポック数
# 損失と正確度，適合率，再現率の履歴を保存するためのdictオブジェクトを用意しておく
history = {'loss':[],'accuracy':[], 'precision':[], 'recall':[], 'test_loss':[], 'test_accuracy':[], 'test_precision':[], 'test_recall':[]}

# 収束が停滞したら学習率を減衰するスケジューラー
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, # オプティマイザーを指定
    mode='max', # 最大値を監視する
    factor=0.1, # 学習率を減衰する割合
    patience=5, # 監視対象のエポック数
    min_lr=0.0001, # 最小学習率
    verbose=True # 学習率を減衰した場合に通知する
)

# 学習を行う
for epoch in range(epochs):
    train_loss, train_acc, train_prec, train_recall = 0., 0., 0., 0. # 訓練1エポックあたりの損失，正確度，適合率，再現率を保持する変数
    test_loss, test_acc, test_prec, test_recall = 0., 0., 0., 0. # 評価1エポックごとの損失，正確度，適合率，再現率を保持する変数

    # 訓練用データセット用のデータローダーを用いて学習する
    for (x, t) in train_dataloader:
        x, t = x.to(device), t.to(device) # torch.Tensorオブジェクトへのデバイスの割り当て
        loss, preds, prec, recall = train_step(x, t) # 上で定義した関数で学習を行い，返り値を得る
        train_loss += loss.item() # 結果の格納
        train_prec += prec.item()
        train_recall += recall.item()
        train_acc += accuracy_score(t.tolist(), preds.argmax(dim=-1).tolist())

    # 評価データセット用のデータローダーを用いて評価する
    for (x, t) in test_dataloader:
        x, t = x.to(device), t.to(device) # torch.Tensorオブジェクトにデバイスを割り当てる
        loss, preds, prec, recall = test_step(x, t) # 上で定義した関数で評価データセットに関しての返り値を得る
        test_loss += loss.item() # 結果の格納
        test_prec += prec.item()
        test_recall += recall.item()
        test_acc += accuracy_score(t.tolist(), preds.argmax(dim=-1).tolist())
        
    # 訓練時の損失，正確度，適合率，再現率の平均値を取得
    avg_train_loss, avg_train_acc, = train_loss / len(train_dataloader), train_acc / len(train_dataloader)
    avg_train_prec, avg_train_recall = train_prec / len(train_dataloader), train_recall / len(train_dataloader)

    # 評価時の損失，正確度，適合率，再現率の平均値を取得
    avg_test_loss, avg_test_acc, = test_loss / len(test_dataloader), test_acc / len(test_dataloader)
    avg_test_prec, avg_test_recall = test_prec / len(test_dataloader), test_recall / len(test_dataloader)

    # 用意しておいたリストに学習用データセットの性能評価結果を保存する
    history['loss'].append(avg_train_loss)
    history['accuracy'].append(avg_train_acc)
    history['precision'].append(avg_train_prec)
    history['recall'].append(avg_train_recall)

    # 用意しておいたリストに評価用データセットの性能評価結果を保存する
    history['test_loss'].append(avg_test_loss)
    history['test_accuracy'].append(avg_test_acc)
    history['test_precision'].append(avg_test_prec)
    history['test_recall'].append(avg_test_recall)
    
    # 1エポックごとに結果を出力(小数点以下4桁を指定している)
    print('epoch({0}) train_loss: {1:.4} train_acc: {2:.4} train_precision: {3:.4} train_recall: {4:.4}'.format(
            epoch+1, avg_train_loss, avg_train_acc, avg_train_prec, avg_train_recall)
    )
    print('epoch({0}) test_loss: {1:.4} test_acc: {2:.4} test_precision: {3:.4} test_recall: {4:.4}'.format(
            epoch+1, avg_test_loss, avg_test_acc, avg_test_prec, avg_test_recall)
    )
    scheduler.step(avg_test_acc) # スケジューラーを用いて性能を監視する

##### 入力7-16

In [ ]:
'''
12. 損失と精度をグラフにする
'''
# 損失に関するグラフを描画
plt.plot(history['loss'], marker='.', label='loss (Training)')
plt.plot(history['test_loss'], marker='.', label='loss (Test)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

# 正解率に関するグラフを描画
plt.plot(history['accuracy'], marker='.', label='accuracy (Training)')
plt.plot(history['test_accuracy'], marker='.', label='accuracy (Test)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()